#### Basic Imports

In [4]:
import numpy as np 
import pandas as pd 
import yfinance as yf 
import datetime
import pyfolio as pf

import warnings
warnings.filterwarnings('ignore')

import cufflinks
from pylab import plt
plt.style.use('seaborn')
pd.set_option('mode.chained_assignment',None)
cufflinks.set_config_file(offline=True)
%config InlineBackend.figure_format='svg'

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

#### Backtest a strategy using three moving average on any indices such as Nifty50,SPY,HSI and so on.
- 1. Compute three moving averages of 20,40 and 80
- 2. Go long when the prices crosses above all three moving averages.
- 3. Exit the long positions when the prices crosses below any of the three moving averages.
- 4. Go short when the price crosses below all three moving averages.
- 5. Exit the short positions when the price crosses above any of the three moving averages
- 6. Optional: Optimise all three moving averages

#### Implementing the strategy using OOP

Financial data Class

In [74]:
class Financial_Data:
    def __init__(self,ticker,start_date,end_date):
        self.ticker=ticker
        self.start_date=start_date
        self.end_date=end_date
        self.get_data()
        self.prepare_data()
        self.plot_data()
        
    def get_data(self):
        self.data_raw=yf.download(tickers=self.ticker,start=self.start_date,end=self.end_date).dropna()
    def prepare_data(self):#during optimization need fresh data everytime so need to store and work on it
        self.data=self.data_raw.copy()
        self.data['bnh_returns']=np.log(self.data['Adj Close']/self.data['Adj Close'].shift(1))
    def plot_data(self):# To check wheather the data is clean or not
        self.data['bnh_returns'].cumsum().iplot(title='BnH_returns')
    

Backtesting Class and optimization within the class

In [147]:
class SMA_Backtesting(Financial_Data):
    def indicators(self):#indicators defining and creating their respective column
        self.data['sma_short']=self.data['Adj Close'].rolling(window=self.sma_short,center=False).mean()
        self.data['sma_mid']=self.data['Adj Close'].rolling(window=self.sma_mid,center=False).mean()
        self.data['sma_long']=self.data['Adj Close'].rolling(window=self.sma_long,center=False).mean()
        self.data.dropna(inplace=True)
    
    def backtest_strategy(self,sma_short,sma_mid,sma_long,cutoff=None):
        if cutoff is None: #Introduction of cuttoff is for optimization (refference-DMP-03)
            cutoff=sma_long
        self.prepare_data() #important while optimization
        self.sma_short=sma_short
        self.sma_mid=sma_mid
        self.sma_long=sma_long
        self.indicators()
        self.data_=self.data.iloc[cutoff:].copy()
        #self.signals()
        #self.position()
        #self.returns()
    
    #def signals(self):   #conditions applied as strategy     
        self.data_['signal']=np.where((self.data_['sma_short']>self.data_['sma_mid'])
                                    &(self.data_['sma_short']>self.data_['sma_long']),1,0)
        self.data_['signal']=np.where((self.data_['sma_short']<self.data_['sma_mid'])
                                    &(self.data_['sma_short']<self.data_['sma_long'])
                                     ,-1,self.data_['signal'])
    
    #def position(self):# To replace zero by previous signals and make the column just with 1/-1
        self.data_['position']=self.data_['signal'].replace(to_replace=0,method='ffill')
    
    #def returns(self):#calculating strategy returns
        self.data_['strategy_returns']=self.data_['position'].shift(1) *self.data_['bnh_returns']
       # perf=self.data_['strategy_returns'].cumsum()[-1]
        #perf= self.data_[['bnh_returns','strategy_returns']].sum().apply(np.exp)
        return self.data_['strategy_returns'].cumsum()[-1]
    
    def plot_analysis(self):
        # A plot to check if the strategy is working as planned:
        self.data_[['sma_short','sma_mid','sma_long','position']].iplot(secondary_y='position'
                                                                       ,title='Checking if postion are generated properly')
        
        # A plot to check how the strategy performs relatively to buy and hold strategy
        self.data_[['bnh_returns','strategy_returns']].cumsum().iplot(title='Buy and Hold v/s crossover strategy cummulative returns')
        
    def optimize_SMA(self,sma_short,sma_mid,sma_long):
        cutoff=max(sma_long)
        self.results=pd.DataFrame()
        for SMA_short,SMA_mid,SMA_long in product(sma_short,sma_mid,sma_long):
            perf=self.backtest_strategy(SMA_short,SMA_mid,SMA_long,cutoff)
            df=pd.DataFrame({'sma_short':SMA_short,'sma_mid':SMA_mid,'sma_long':SMA_long},index=[0])
            self.results=self.results.append(df,ignore_index=True)                            

In [148]:
#inputs
end_date=datetime.date(2021,7,1)
start_date=end_date-pd.Timedelta(days=3*252)

In [142]:
from itertools import product #Inputs for optimization can change the values of list
sma_short=[10,20,30]
sma_mid=[40,80,160]
sma_long=[80,180,280]
list(product(sma_short,sma_mid,sma_long))

[(10, 40, 80),
 (10, 40, 180),
 (10, 40, 280),
 (10, 80, 80),
 (10, 80, 180),
 (10, 80, 280),
 (10, 160, 80),
 (10, 160, 180),
 (10, 160, 280),
 (20, 40, 80),
 (20, 40, 180),
 (20, 40, 280),
 (20, 80, 80),
 (20, 80, 180),
 (20, 80, 280),
 (20, 160, 80),
 (20, 160, 180),
 (20, 160, 280),
 (30, 40, 80),
 (30, 40, 180),
 (30, 40, 280),
 (30, 80, 80),
 (30, 80, 180),
 (30, 80, 280),
 (30, 160, 80),
 (30, 160, 180),
 (30, 160, 280)]

In [9]:
fd=Financial_Data('^NSEI',start_date,end_date)

[*********************100%***********************]  1 of 1 completed


In [143]:
sma=SMA_Backtesting('^NSEI',start_date,end_date)

[*********************100%***********************]  1 of 1 completed


In [129]:
sma.backtest_strategy(20,50,80)

0.5483295666664583

In [123]:
sma.plot_analysis()

In [124]:
sma.returns()


AttributeError: 'SMA_Backtesting' object has no attribute 'returns'

In [149]:
sma.optimize_SMA(sma_short,sma_mid,sma_long)

IndexError: index -1 is out of bounds for axis 0 with size 0

In [120]:
sma.results.sort_values('STRAT',ascending=False)

,sma_short,sma_mid,sma_long,BENCH,STRAT
0,10,40,80,1.221163,1.112331
21,30,80,80,1.221163,1.106813
18,30,40,80,1.221163,1.106813
7,10,160,180,1.075500,1.094854
25,30,160,180,1.075500,1.094854
22,30,80,180,1.075500,1.094854
16,20,160,180,1.075500,1.094854
13,20,80,180,1.075500,1.094854
4,10,80,180,1.075500,1.094854
12,20,80,80,1.221163,1.085235
